# Using Physics-Informed Neural Network to solve the 1d heat equation

In the following example, we will solve the one-dimensional heat equation using a physics-informed neural network (PINN) implemented in TensorFlow. The problem definition can be interpreted as **cooling a one-dimensional rod** from an initial temperature distribution $u(x,0)=f(x)$. The governing partial differential equation (PDE) with the corresponding initial (IC) and boundary (BC) condition are given in **non-dimensionalized form** by:

* PDE: $\frac{\partial u(x,t)}{\partial t} - \frac{\partial^2 u(x,t)}{\partial x^2}=0$,
*  IC: $u(x,0)=f(x), \quad\quad\quad\quad 0<x<1$, 
*  BC: $u(0,t)=u(1,t)=0, \quad\quad    t>0$.

We start by loading some modules

In [ ]:
import numpy as np
import tensorflow as tf

### Creating Physics-Informed Neural Network

We proceed by approximating the solution function $u(x,t)$ with a fully-connected neural network function $u_\theta(x,t)$. The input-dimension will be set to two ($x,t$), the output dimension to one ($u$).

In [ ]:
from src.neural_network import NeuralNetwork
PINN = NeuralNetwork(n_hidden=4, n_neurons=50, activation='tanh')

As a next step, we will construct the PDE (definition given above) and define the **Physics Loss** $L_F$. We can use the Tensorflow [tf.GradientTape()](https://www.tensorflow.org/api_docs/python/tf/GradientTape) function to obtain the partial derivatives $\frac{\partial u_\theta}{\partial t}$ and $\frac{\partial^2 u_\theta}{\partial x^2}$ with automatic differentiation.

In [ ]:
def get_loss_F(X_col):
   
    # tape forward propergation to retrieve gradients
    with tf.GradientTape() as t:
        t.watch(X_col)
        with tf.GradientTape() as tt:
            tt.watch(X_col)
            U = PINN(X_col)
        U_d = tt.batch_jacobian(U, X_col)        
    U_dd = t.batch_jacobian(U_d, X_col)

    # U_d shape: (n_col, u_i, dx_i)
    u_t = U_d[:, 0, 1]
    # U_dd shape: (n_col, u_i, dx_i, dx_j)
    u_xx = U_dd[:, 0, 0, 0]

    # Heat diffusion equation
    res_F = u_t - u_xx
    
    # determine residual errors 
    loss_F = tf.reduce_mean(tf.square(res_F))

    return loss_F

For the **Data Loss** we simply use the MSE.

In [ ]:
def get_loss_u(X, u):
    u_pred = PINN(X)     
    return tf.reduce_mean(tf.square(u_pred - u))

Next, we implement the training step. To define the **multi-objective loss** we use the (unweighted) **linear combination of both losses** - motivated persons could play around with weighting both losses differently.

In [ ]:
def train_step(optimizer, X, u, X_col):
    
    # open a GradientTape to record forward/loss pass                   
    with tf.GradientTape() as tape:  
        loss_u = get_loss_u(X, u)
        loss_F = get_loss_F(X_col)
                
        # Linear combination of data and physics loss
        loss = loss_u + loss_F


    # gradients PDE fit
    grads = tape.gradient(loss, PINN.trainable_weights)                    
    # perform single GD step 
    optimizer.apply_gradients(zip(grads, PINN.trainable_weights))
    
    return loss_u, loss_F

## Creating datasets for IC, BC and collocation points

We start by defining the initial temperature distribution $f(x)$ for the **initial condition** (IC). To measure the model prediction against a test set, we use $f(x)=sin(\pi x)$ as an initial temperature distribution (since the solution for this problem is given analytically and will be implemented below).

In [ ]:
f = lambda x: np.sin(np.pi*x)

Next, we create an input dataset $\{x^{(i)},0\}_{i=1}^{N_{IC}}$ with the corresponding temperature values and convert it to a tensorflow tensor. **Note: $x\in[0,1]$**.

In [ ]:
N_IC = 100
### Sample data from IC ###
x_IC = np.random.rand(N_IC)
u_IC = f(x_IC)

# Input data has to be converted to tf.tensors
X_IC = tf.convert_to_tensor([[x, 0] for x in x_IC], dtype=tf.float32)
u_IC = tf.convert_to_tensor([[u] for u in u_IC], dtype=tf.float32)

For the **boundary condition** (BC) we sample time values and construct the input datasets $\{0, t^{(i)}\}_{i=1}^{N_{BC}}$ and $\{1, t^{(i)}\}_{i=1}^{N_{BC}}$. Here, we **limit the computational domain** of the time coordinate to $t\in[0,0.5]$.

In [ ]:
N_BC = 100
### Sample data from BC ###
X_bot = [[0, t] for t in np.random.rand(N_BC)*0.5]
X_top = [[1, t] for t in np.random.rand(N_BC)*0.5]
u_bot = [[0] for _ in range(N_BC)]
u_top = [[0] for _ in range(N_BC)]

# Input data has again to be converted to tf.tensors
X_BC = tf.convert_to_tensor(X_bot + X_top, dtype=tf.float32)
u_BC = tf.convert_to_tensor(u_bot + u_top, dtype=tf.float32)

Now, we **plot** the IC and BC datasets and concatenate them to a **single training dataset**.

In [ ]:
from src.plots import plot_input_data
plot_input_data(X_IC, u_IC, X_BC, u_BC)

# Concatenate to single training data set
X = tf.concat([X_IC, X_BC], axis=0)
u = tf.concat([u_IC, u_BC], axis=0)

As a next, important step, we sample the **collocation points** from inside the function domain using **latin-hypercube sampling** found in the pyDOE module. The sampled domain will be $x,t\in[0,1]\times[0,0.5]$.

In [ ]:
from pyDOE import lhs
N_col = 2500
### Sample collocation points ###
X_col = tf.convert_to_tensor([1, 0.5] * lhs(2, N_col), dtype=tf.float32)

Finally, to test our model against a test set, we define the analytical solution for our problem definition which is given by $u(x,t)=sin(\pi x)\cdot exp(-\pi^2 t)$. (**CAUTION**: the analytical solution will we only valid for $f(x)=sin(\pi x)$ and $u(0,t)=u(1,t)=1$ with $t>0$).

In [ ]:
def get_u_solution(X):
    x = X[:,0]
    t = X[:,1]
    u_sol = np.sin(np.pi*x) * np.exp(-np.pi**2*t)
    return tf.expand_dims(u_sol, axis=1)

We sample test points inside the function domain and get the corresponding temperature values.

In [ ]:
N_test = 1000
### Sample collocation points ###
X_test = tf.convert_to_tensor([1, 0.5] * lhs(2, N_test), dtype=tf.float32)
u_test = get_u_solution(X_test)

## PINN Training (PDE Solving)

The last step we have to do, is implementing the training loop which calls the *train_step()* function.
Here, we use the Adam optimizer in full-batch mode. The initial learning rate is set to $1e-3$ which should be ok for this basic test. Learning rate schedule, e.g. exponential decay, could be used to further optimize and finetune the model training. Number of epochs can be changed at will. (**Note**: for an enormous run time speed up, we can use [tf.function()](https://www.tensorflow.org/api_docs/python/tf/function) on the *train_step()* function which converts it into a Tensorflow graph function).

In [ ]:
learning_rate = 1e-03

# Adam optimizer with default settings for momentum
optimizer = tf.keras.optimizers.Adam(learning_rate)  
# convert function to tf.function (graph function)
train_step_tf = tf.function(train_step)

**Executing the following cell will start the PINN training**

In [ ]:
n_epochs = 5000

for epoch in range(n_epochs): 
    
    loss_u, loss_F = train_step_tf(optimizer, X, u, X_col)
    
    if (epoch % 100 == 0):
        print(f'Epoch {epoch:<5} || Loss_u: {loss_u:1.2e} | Loss_F: {loss_F:1.2e}')
        
print("### Finished training ###")

To check how well the model was doing in solving the PDE, we plot the model prediction over the entire function domain.

In [ ]:
from src.plots import plot_prediction
plot_prediction(PINN)

If we have used the *default* problem definition, we can also check the model's accucary by using the test set. A common performance measure is the **relative L2 Error** given by $rel. L^2 = ||u_{pred}-u_{true}||/||u_{true}||$

In [ ]:
u_pred = PINN(X_test)
rel_L2 = np.linalg.norm(u_pred-u_test)/np.linalg.norm(u_test)
print(f"rel. L2 Error: {rel_L2*100:1.3f}%")